<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [1]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [2]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print(DATA_DIR)
print(PARQUET_DIR)
print('Environment ready.')

Datasets
data_parquet
Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [7]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [8]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "tx_id": np.int32,              # seconda colonna: contatore 1…10 572 827
    "block_height": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "has_witness": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}

columns_transactions = [
    "timestamp", "tx_id", "block_height",
     "has_witness", "fee"
]

## Caricamento dei Dataset

In [5]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,dtype=dtype_mapping)

## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [9]:
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.walletexplorer.com/",
}

In [ ]:
def next_page(driver):
    #cerco il link alla prossima pagina
    for _ in ("Next","Last"):
        url = find_next_link(driver,_)
        if url:return url
    return None
        
def find_next_link(driver,element):
    try:
        el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{element}")]')
        url = el.get_attribute("href")
        el.click()
        return url
    except:
        return None

# def get_transactions_link(soup,class_):
#     transactions = soup.find_all('tr',class_=class_)
#     page_links = []
#     for trans in transactions:
#         txid = trans.find('td',class_="txid").find('a')['href']
#         page_links.append(txid)
#     return page_links

# def safe_get(session, url, tries=3, delay=1):
    for attempt in range(tries):
        try:
            return session.get(url, timeout=10)
        except (requests.exceptions.ConnectionError,
                requests.exceptions.RemoteDisconnectedError):
            if attempt == tries - 1:
                raise
            time.sleep(delay * (attempt + 1))  # 1s, poi 2s, poi 3s…

def scrape_step(driver):
    #recupero le entries
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # seleziona tutti gli elementi che hanno l’attributo data-wallet-id
    wallet_elements = soup.select('[data-wallet-id]')
    # estrai il valore dell’attributo
    wallet_ids = [el['data-wallet-id'] for el in wallet_elements]
    #vado alla prossima pagina
    return wallet_ids




### Definizione Funzione per Scraping

In [ ]:
def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet tramite form
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()
    
    #mettere funzione di selenium
    time.sleep(1)

    try:
        while True:
            #recupero di tutti gli input/output della pagina
            yield from scrape_step(driver)

            if next_page(driver) == None:break;
    except:
            print('Something went wrong')
    finally:
        driver.close()    
   


### Probabilmente non serve ma si cancella solo a progetto finito

In [ ]:
# BLOCK_RE = re.compile(r"\d+")
# FEE_RE = re.compile(r"\d+(?:\.\d+)?")

# def get_info_from_transaction_table(trans_table_link,session):
#     # res = session.get(f'{"https://www.walletexplorer.com"}{trans_table_link}',timeout=10)
#     res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
#     soup = BeautifulSoup(res.text,"html.parser")
#     cells = soup.select('table.info td')
#     all_txt = [td.get_text() for td in cells]
#     match  = BLOCK_RE.search(all_txt[1])
#     if match:blockId = int(match.group())
#     else:   blockId =  None
#     match_2 = FEE_RE.search(all_txt[4]) 
#     if match_2: fee = float(match_2.group())  
#     else: fee= None
#     return {
#         "timestamp":all_txt[2],
#         "blockId":blockId,
#         "txId":all_txt[0],
#         "isCoinbase":0,
#         "fee":fee
#     }


# def get_wallet_adress(trans_table_link,session):
#     res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
#     soup = BeautifulSoup(res.text,"html.parser")
#     cells = soup.select('table.info td')
#     all_txt = [td.get_text() for td in cells]
#     return all_txt[3]

# def get_all_trans(transaction_list):
#     infos = []
#     session = requests.Session()
#     session.headers.update(headers)
#     for transaction_link in transaction_list:
#         # info = get_info_from_transaction_table(transaction_link,session)
#         info = get_wallet_adress(transaction_link,session)
#         infos.append(info)
#     return infos

<center>

# Analisi Transazioni DiceOnCrack


In [ ]:
diceoncrack_transaction = list(scrape_wallet_explorer('DiceonCrack'))
print(len(diceoncrack_transaction))

91321


In [48]:
display(diceoncrack_transaction)

['00cf1613d74c513e',
 '99df98abccf37c07',
 'c2007ccf096138f9',
 '77bebf6682d16a66',
 '3383c3c7d2d376c4',
 '8aa9b2e64b8d83dd',
 '7d90100a1ac56ced',
 'ec9ac79b3a4d00fe',
 '11fe3742882d50cc',
 '5c814aaefb1bcbe4',
 'f24f62123c567576',
 '166ee82c89ad40c4',
 '166ee82c89ad40c4',
 'c9dffbdf043464c7',
 '0008a70ca8bbe3c5',
 '0008a70ca8bbe3c5',
 '0008a70ca8bbe3c5',
 '3b7aff74efe08f94',
 'add44697f4dbbde7',
 '8ff0a5158cb525b3',
 '013b2a52a8979274',
 '41aa1cd065d5fe37',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '5f2b26ed990121c2',
 '5f2b26ed990121c2',
 '00a376a54df460fe',
 'c473658a837249a8',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '022d9ac4502b9ccb',
 '022d9ac4502b9ccb',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '1172596d83a16e70',
 '002c5c9ea45763f9',
 '002c5c9ea45763f9',
 '002c5c9ea45763f9',
 '0c4f1c38b96d10ac',
 '002c5c9ea45763f9',
 '0c4f1c38b96d10ac',
 'd5543daeccdca46e',
 '002c5c9ea45763f9',
 '002c5c9ea45

### Salvataggio delle transazioni in Csv per velocizzare il testing

In [61]:
# salva il DataFrame in un CSV nella cartella corrente
doc_df = pd.DataFrame(diceoncrack_transaction,columns=["inputs/outputs"]).set_index("inputs/outputs")
doc_df.to_csv("diceoncrack.csv", index=True, encoding="utf-8")

### Recupero dei dati dal csv

In [50]:
dice_df = pd.read_csv('diceoncrack.csv',index_col="inputs/outputs")

<center>

# Analisi Transazioni di DeepBit.Net

In [ ]:
deepbit_transactions = list(scrape_wallet_explorer('DeepBit.net'))
print(len(deepbit_transactions))

Something went wrong


In [ ]:
display(deepbit_transactions)

40200
